In [1]:
import nltk
import pandas as pd
from sklearn.metrics import classification_report, f1_score, accuracy_score
import numpy as np

# importing the requests library
import requests
import pickle
import re

In [2]:
df = pd.read_csv('./gold_output.csv')

In [3]:
df

,Unnamed: 0,id,created_at,tweet,sum,true_value,polarity_api_output,is_same,concept_api_output,intensity_api_output,aspect_api_output,scores,compound,comp_score,scores_text_blob,comp_score_text_blob,score_flair,comp_score_flair
0,0,1369812663623180288,2021-03-11 6:19:23,In great news my senior parents are getting th...,2.0,POSITIVE,POSITIVE,1,"['great news', 'senior', 'vaccine', 'thank', '...",0.769,"[('getting', 'POSITIVE'), ('whew', 'POSITIVE')]","{'neg': 0.0, 'neu': 0.575, 'pos': 0.425, 'comp...",0.8625,POSITIVE,0.800000,POSITIVE,0.9949,POSITIVE
1,1,1354514776534220804,2021-01-28 1:11:03,Our Co-Founder and CEO <USER> recently receive...,2.0,POSITIVE,NEGATIVE,0,"['founder', 'recent', 'covid', 'vaccine', 'vac...",-0.821,"[('vaccine', 'POSITIVE'), ('must', 'POSITIVE')...","{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...",0.9230,POSITIVE,0.516667,POSITIVE,0.7248,POSITIVE
2,2,1349045265945264130,2021-01-12 22:57:10,"""This is all we have left. This can't fail."" <...",0.2,POSITIVE,POSITIVE,1,"['fail', 'make', 'vaccine', 'priority', 'plus'...",0.814,"[('mass', 'POSITIVE'), ('ANNOYANCE', 'POSITIVE...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,POSITIVE,-0.250000,NEGATIVE,-0.9779,NEGATIVE
3,3,1366793773728980992,2021-03-02 22:23:24,Dear #MAGA Trump NEVER cared about you. He Jim...,-1.2,NEGATIVE,POSITIVE,0,"['cared', 'stop', 'punch', 'plane', 'traitor',...",0.769,"[('drink', 'NEGATIVE'), ('plan', 'POSITIVE')]","{'neg': 0.183, 'neu': 0.719, 'pos': 0.098, 'co...",-0.3013,NEGATIVE,0.000000,POSITIVE,-0.9990,NEGATIVE
4,4,1371004616897269763,2021-03-14 13:15:47,I am all for risk triaging and in turn getting...,-1.1,NEGATIVE,POSITIVE,0,"['health care', 'risk', 'turn', 'vaccine', 'va...",0.769,"[('getting', 'POSITIVE'), ('more', 'POSITIVE')...","{'neg': 0.042, 'neu': 0.841, 'pos': 0.117, 'co...",0.5789,POSITIVE,0.291667,POSITIVE,-0.9210,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1358616738389106690,2021-02-08 8:50:46,"#LATEST The first batch of 300,000 doses of #C...",0.6,POSITIVE,POSITIVE,1,"['latest', 'batch', 'dose', 'covid', 'vaccine'...",0.769,"[('first', 'NEGATIVE')]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,POSITIVE,0.283333,POSITIVE,-0.9806,NEGATIVE
496,496,1354707290222432257,2021-01-28 13:56:01,"""Rich countries jumping the queue for vaccines...",-1.4,NEGATIVE,POSITIVE,0,"['rich', 'queue', 'vaccine', 'chief', 'warns',...",0.859,"[('read', 'POSITIVE'), ('jumping', 'NEGATIVE')]","{'neg': 0.068, 'neu': 0.753, 'pos': 0.179, 'co...",0.7430,POSITIVE,0.152083,POSITIVE,-0.9348,NEGATIVE
497,497,1357290740712427521,2021-02-04 17:01:44,If one of the symptoms of #Covid was that your...,-1.4,NEGATIVE,POSITIVE,0,"['up for', 'covid', 'dick', 'see', 'everywhere...",0.769,"[('covid', 'NEGATIVE'), ('line', 'POSITIVE')]","{'neg': 0.086, 'neu': 0.914, 'pos': 0.0, 'comp...",-0.5106,NEGATIVE,0.000000,POSITIVE,-0.9999,NEGATIVE
498,498,1370435414306213893,2021-03-12 23:33:58,Who would like this <USER> guide written by th...,0.8,POSITIVE,POSITIVE,1,"['guide', 'brilliant', 'highest', 'wins', 'pro...",0.769,"[('like', 'POSITIVE')]","{'neg': 0.0, 'neu': 0.758, 'pos': 0.242, 'comp...",0.8974,POSITIVE,0.687500,POSITIVE,0.9970,POSITIVE


In [4]:
y_true = df['true_value'].to_numpy()
y_pred_vader = df['comp_score'].to_numpy()
y_pred_sentic = df['polarity_api_output'].to_numpy()
y_pred_text_blob = df['comp_score_text_blob'].to_numpy()
y_pred_flair = df['comp_score_flair'].to_numpy()

y_score_sentic = df['intensity_api_output'].to_numpy()
y_score_vader = df['compound'].to_numpy()
y_score_text_blob = df['scores_text_blob'].to_numpy()
y_score_flair = df['score_flair'].to_numpy()

In [5]:
def calc_label(y):
    x = y.copy()
    x[x=="POSITIVE"] = int(1)
    x[x=="NEGATIVE"] = int(-1)
    return x.astype('int64')

In [6]:
def calc_ensemble(y1, y2, y3, y4):
    x1, x2, x3, x4 = y1.copy(), y2.copy(), y3.copy(), y4.copy()
    ans = np.zeros(y1.shape)
#     x1 = (x1-np.mean(x1))/(np.std(x1))
#     x2 = (x2-np.mean(x2))/(np.std(x2))
#     x3 = (x3-np.mean(x3))/(np.std(x3))
#     x4 = (x4-np.mean(x4))/(np.std(x4))
    x1 = 0.82552037*x1
    x2 = 1.46025455*x2
    x3 = 0.87687926*x3
    x4 = 0.67111069*x4
    ans[1/(1+np.exp(-(x1+x2+x3+x4 + 0.02344407)))>=0.5] = 1
    ans[1/(1+np.exp(-(x1+x2+x3+x4 + 0.02344407)))<0.5] = -1
#     ans[(x1+x2+x3+x4+0.23781291)>=0] = 1
#     ans[(x1+x2+x3+x4+0.23781291)<0] = -1
    return ans.astype('int64')

In [7]:
print("F1:", f1_score(calc_label(y_true), calc_label(y_pred_vader)),
     "Acc:", accuracy_score(calc_label(y_true), calc_label(y_pred_vader)))

F1: 0.8634482758620691 Acc: 0.802


In [8]:
print("F1:", f1_score(calc_label(y_true), calc_ensemble(y_score_sentic,
                                                        y_score_vader,
y_score_text_blob,
y_score_flair)))
print("Acc:", accuracy_score(calc_label(y_true), calc_ensemble(
y_score_sentic,
    y_score_vader,
y_score_text_blob,
y_score_flair)))

F1: 0.8781204111600588
Acc: 0.834


In [9]:
cnt = 0
for i in range(y_true.shape[0]):
    flag = 0
#     if (y_true[i] == y_pred_vader[i]):
#         flag += 1
#     if (y_true[i] == y_pred_sentic[i]):
#         flag += 1
#     if (y_true[i] == y_pred_text_blob[i]):
#         flag += 1
#     if (y_true[i] == y_pred_flair[i]):
#         flag += 1
    tot = y_score_vader[i]+y_score_sentic[i]+y_score_text_blob[i]+y_score_flair[i]
    ans = "NEGATIVE"
    if (tot >=0):
        ans = "POSITIVE"
    if (y_true[i] != ans):
#     if flag == 3:
        cnt += 1
        print(df['tweet'].iloc[i], tot)
        print(y_true[i], y_score_sentic[i], y_score_vader[i], y_score_text_blob[i], y_score_flair[i])
        print(df['aspect_api_output'].iloc[i])
print(cnt)

"This is all we have left. This can't fail." <USER>says America needs to make mass vaccinations its #1 priority. Plus, Can you spread #COVID19 even after 2 #vaccine doses?" Find out here <EMOJI:backhand_index_pointing_down><EMOJI:light_skin_tone> -0.41389999999999993
POSITIVE 0.8140000000000001 0.0 -0.25 -0.9779
[('mass', 'POSITIVE'), ('ANNOYANCE', 'POSITIVE'), ('america', 'NEGATIVE')]
I am all for risk triaging and in turn getting vaccinated in due course. But why has <USER> had two more vaccines more than basically every health care worker I know? <EMOJI:thinking_face> #auspol #vaccine #COVID19Aus 0.7185666666666669
NEGATIVE 0.769 0.5789 0.2916666666666667 -0.9209999999999999
[('getting', 'POSITIVE'), ('more', 'POSITIVE'), ('than', 'POSITIVE')]
Mags says My son ,who is a nurse ,and some of his colleagues had their first vaccine before Christmas ,everyone who had the vaccine were later ill and tested positive for #covid , all the other nurses who said no to the #vaccine were all ok !!

In [10]:
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

clfs = [
    Perceptron(tol=1e-8, random_state=0),
    LogisticRegression(),
    SVC(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    xgb.XGBClassifier(random_state=0),
    BaggingClassifier(),
    RandomForestClassifier(random_state = 0)
]

# clf = Perceptron(tol=1e-8, random_state=0)
# clf = LogisticRegression()
# clf = SVC()
# clf = AdaBoostClassifier()
# clf = GradientBoostingClassifier()
# clf = BaggingClassifier()
# clf = xgb.XGBClassifier(random_state=0)
# clf = RandomForestClassifier(random_state = 0)


In [11]:
X = np.vstack([y_score_sentic, y_score_vader, y_score_text_blob, y_score_flair])
X = X.T
y = calc_label(y_true)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [13]:
X_train.shape, X_test.shape

((400, 4), (100, 4))

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
for clf in clfs:
    print('-'*20)
    print(clf)
    clf.fit(X_train, y_train)
    print(clf.score(X_train, y_train))
    print("F1:", f1_score(calc_label(y_test), clf.predict(X_test)),
     "Acc:", accuracy_score(calc_label(y_test), clf.predict(X_test)))

--------------------
Perceptron(tol=1e-08)
0.5425
F1: 0.5862068965517242 Acc: 0.52
--------------------
LogisticRegression()
0.805
F1: 0.8368794326241135 Acc: 0.77
--------------------
SVC()
0.8375
F1: 0.8684210526315789 Acc: 0.8
--------------------
AdaBoostClassifier()
0.9175
F1: 0.8783783783783784 Acc: 0.82
--------------------
GradientBoostingClassifier()
0.9775
F1: 0.8843537414965987 Acc: 0.83
--------------------
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, reg_alpha=None, reg_lambda=None,
              scale_p

In [16]:
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [17]:
clf.score(X_train, y_train)

1.0

In [18]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}

In [19]:
print("F1:", f1_score(calc_label(y_test), clf.predict(X_test)),
     "Acc:", accuracy_score(calc_label(y_test), clf.predict(X_test)))

F1: 0.9054054054054055 Acc: 0.86


In [20]:
print(clf.coef_, clf.intercept_)

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'